## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from tab_dataset.cdataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, fields, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le dict 'fields'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(fields)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
    # 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
    # log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    #'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  79418


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
     { "name": "index",
       "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 4 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  3218 soit :  0.04051978140975598
nombre de pdc en itinerance   :  76200


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (76200)
   0 : index (0 - 76200)
   1 : contact_operateur (76050 - 150)
   2 : nom_enseigne (72499 - 3701)
   3 : coordonneesXY (54019 - 22181)
   4 : adresse_station (54401 - 21799)
   5 : id_station_itineranc (43812 - 32388)
   6 : nom_station (53729 - 22471)
   7 : implantation_station (76195 - 5)
   8 : nbre_pdc (76142 - 58)
   9 : condition_acces (76198 - 2)
   10: horaires (75649 - 551)
   11: station_deux_roues (76194 - 6)
   12: id_pdc_itinerance (4918 - 71282)
   13: date_maj (64683 - 11517)
   14: last_modified (75696 - 504)


-----------------------
## 4 - Bilan initial intégrité
- 18 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           9834
contact_operateur - id_station_itinerance           6230
nom_enseigne - id_station_itinerance                6843
coordonneesXY - id_station_itinerance               8699
id_station_itinerance - id_pdc_itinerance           1086
nom_station - id_station_itinerance                 6418
implantation_station - id_station_itinerance        2826
nbre_pdc - id_station_itinerance                    7477
condition_acces - id_station_itinerance               24
horaires - id_station_itinerance                      74
station_deux_roues - id_station_itinerance          7502
adresse_station - coordonneesXY                     2960

nombre d'enregistrements sans erreurs :  62129
nombre d'enregistrements avec au moins une erreur :  14071
taux d'erreur :  18  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  4918  soit  35  %
nombre de pdc sans doublon   :  9153  soit  65  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            191
nom_enseigne - id_station_itinerance                 194
coordonneesXY - id_station_itinerance               1110
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  735
implantation_station - id_station_itinerance         402
nbre_pdc - id_station_itinerance                     697
condition_acces - id_station_itinerance               17
horaires - id_station_itinerance                      51
station_deux_roues - id_station_itinerance           521
adresse_station - coordonneesXY                     2423

nombre d'enregistrements sans erreurs :  4786
nombre d'enregistrements avec au moins une erreur :  4367


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1130  soit  8  %
nombre de pdc sans doublon   :  3237


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             22
nom_enseigne - id_station_itinerance                  25
coordonneesXY - id_station_itinerance                401
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  231
implantation_station - id_station_itinerance         107
nbre_pdc - id_station_itinerance                     256
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      27
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     1867

nombre d'enregistrements sans erreurs :  233
nombre d'enregistrements avec au moins une erreur :  3004  soit  69  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (3004)
   0 : index (0 - 3004)
   1 : contact_operateur (2961 - 43)
   2 : nom_enseigne (2835 - 169)
      9 : condition_acces (167 - 2)
   3 : coordonneesXY (2218 - 786)
   4 : adresse_station (1759 - 1245)
   5 : id_station_itineranc (1119 - 1885)
      13: date_maj (1427 - 458)
   6 : nom_station (1771 - 1233)
   7 : implantation_station (2999 - 5)
   8 : nbre_pdc (2983 - 21)
   10: horaires (2957 - 47)
   11: station_deux_roues (2999 - 5)
   12: id_pdc_itinerance (0 - 3004)
   14: last_modified (2930 - 74)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  3  erreurs :  109


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [14]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3004
total des doublons à supprimer :  6048
nombre de pdc avec controles ok :  67148


In [15]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [16]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [17]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (67148)
   4 : id_station_itineranc (36729 - 30419)
      1 : nom_enseigne (26862 - 3557)
      2 : coordonneesXY (10268 - 20151)
         0 : contact_operateur (20017 - 134)
         3 : adresse_station (755 - 19396)
      5 : nom_station (10285 - 20134)
      6 : implantation_station (30414 - 5)
      7 : nbre_pdc (30361 - 58)
      8 : condition_acces (30417 - 2)
      9 : horaires (29907 - 512)
      10: station_deux_roues (30413 - 6)
      19: paiement_autre (30411 - 8)
   11: id_pdc_itinerance (0 - 67148)
   12: puissance_nominale (67053 - 95)
   13: prise_type_ef (67140 - 8)
   14: prise_type_2 (67141 - 7)
   15: prise_type_combo_ccs (67141 - 7)
   16: prise_type_chademo (67141 - 7)
   17: prise_type_autre (67141 - 7)
   18: paiement_acte (67140 - 8)
   20: reservation (67141 - 7)
   21: accessibilite_pmr (67144 - 4)
   22: restriction_gabarit (67048 - 100)
   23: date_maj (56505 - 10643)
   24: last_modified (66711 - 437)


### Indicateurs

In [18]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [19]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  73  % des écarts

sav@izivia.com 758
supervision-ev.france@totalenergies.com 554
support@alizecharge.fr 364
info@ionity.eu 326
web@freshmile.com 192


In [20]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           2121
71  %


In [21]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [22]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
62,353,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239151,2022-11-03,2023-12-14T14:56:18.422000+00:00
63,354,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239152,2022-11-03,2023-12-14T14:56:18.422000+00:00
64,355,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239153,2022-11-03,2023-12-14T14:56:18.422000+00:00
97,516,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2023-12-14T14:56:18.422000+00:00
98,517,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2023-12-14T14:56:18.422000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,72361,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008379,2023-10-17,2023-12-14T14:57:07.770000+00:00
2686,72362,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008380,2023-10-17,2023-12-14T14:57:07.770000+00:00
2687,72363,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008381,2023-10-17,2023-12-14T14:57:07.770000+00:00
2688,72364,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008382,2023-10-17,2023-12-14T14:57:07.770000+00:00


In [23]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
97,516,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2023-12-14T14:56:18.422000+00:00
98,517,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2023-12-14T14:56:18.422000+00:00
99,518,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553331,2022-11-03,2023-12-14T14:56:18.422000+00:00
100,519,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553341,2022-11-03,2023-12-14T14:56:18.422000+00:00
101,520,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553351,2022-11-03,2023-12-14T14:56:18.422000+00:00
102,521,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553361,2022-11-03,2023-12-14T14:56:18.422000+00:00
103,522,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628411,2022-11-03,2023-12-14T14:56:18.422000+00:00
104,523,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628421,2022-11-03,2023-12-14T14:56:18.422000+00:00
105,524,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628431,2022-11-03,2023-12-14T14:56:18.422000+00:00
106,525,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628441,2022-11-03,2023-12-14T14:56:18.422000+00:00


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [24]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print('\n', sum(itinerance_4.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 760
Répartition des incohérences  sav@izivia.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                  0
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                    0
implantation_station - id_station_itinerance           0
nbre_pdc - id_station_itinerance                       0
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      758

 558
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance     

### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [25]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
42,322,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-12-14T14:56:18.422000+00:00
43,323,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-12-14T14:56:18.422000+00:00
44,324,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-12-14T14:56:18.422000+00:00
45,325,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-12-14T14:56:18.422000+00:00
674,28201,support@iecharge.io,Vezin-le-Coquet,"[-1.7547021, 48.1252516]","4001 Rue de la Belle-Epine, 35132 Vezin-le-Coquet",FRIENE35353A,Vezin-le-Coquet,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIENE35353A1,2024-01-15,2024-01-16T00:00:17.176000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560,66773,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès libre,24/7,false,FRSHEE944,2024-01-15,2024-01-16T00:00:17.176000+00:00
2561,66774,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE951,2024-01-15,2024-01-16T00:00:17.176000+00:00
2562,66775,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE952,2024-01-15,2024-01-16T00:00:17.176000+00:00
2563,66776,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE953,2024-01-15,2024-01-16T00:00:17.176000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [26]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
47,330,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2023-12-14T14:56:18.422000+00:00
48,331,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2023-12-14T14:56:18.422000+00:00
49,332,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2023-12-14T14:56:18.422000+00:00
50,333,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2023-12-14T14:56:18.422000+00:00
51,334,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2023-12-14T14:56:18.422000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,72364,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008382,2023-10-17,2023-12-14T14:57:07.770000+00:00
2697,72373,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",28 bd. Jeanne d’Arc,FRTCBP01835,AMP | 128 boulevard Jeanne d'Arc,Voirie,2,Accès libre,24/7,FALSE,FRTCBE008393,2023-10-17,2023-12-14T14:57:07.770000+00:00
2698,72374,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",28 bd. Jeanne d’Arc,FRTCBP01836,AMP | 128 boulevard Jeanne d'Arc,Voirie,2,Accès libre,24/7,FALSE,FRTCBE008394,2023-10-17,2023-12-14T14:57:07.770000+00:00
2705,72381,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",59 rue Niels Bohr,FRTCBP01836,AMP | 59 rue Niels Bohr,Voirie,2,Accès libre,24/7,FALSE,FRTCBE008407,2023-10-17,2023-12-14T14:57:07.770000+00:00


### Coherence adresse - coordonnees
- 1807 pdc ont une adresse non cohérente avec les coordonnées géographiques

Toutes les stations Ionity en erreur (326) ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

In [27]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
679,28510,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2023-12-14T14:56:35.115000+00:00
680,28511,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2023-12-14T14:56:35.115000+00:00
681,28512,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2023-12-14T14:56:35.115000+00:00
682,28513,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2023-12-14T14:56:35.115000+00:00
683,28516,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2023-12-14T14:56:35.115000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,29329,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440316,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440316,2024-01-15,2024-01-16T00:00:17.176000+00:00
1001,29330,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440317,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440317,2024-01-15,2024-01-16T00:00:17.176000+00:00
1002,29331,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440318,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440318,2024-01-15,2024-01-16T00:00:17.176000+00:00
1003,29332,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE44035,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE44035,2024-01-15,2024-01-16T00:00:17.176000+00:00


In [28]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
680,28511,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2023-12-14T14:56:35.115000+00:00
727,28650,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126401,2024-01-15,2024-01-16T00:00:17.176000+00:00
728,28651,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126402,2024-01-15,2024-01-16T00:00:17.176000+00:00
729,28652,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126404,2024-01-15,2024-01-16T00:00:17.176000+00:00
730,28653,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126405,2024-01-15,2024-01-16T00:00:17.176000+00:00
731,28654,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE12645,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE12645,2024-01-15,2024-01-16T00:00:17.176000+00:00


In [29]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
725,28617,info@ionity.eu,IONITY GMBH,"[4.732409, 44.194832]","Aire de Mornas Village , A7, 84550 Mornas",FRIONE440300,IONITY Mornas Village,Station dédiée à la recharge rapide,16,Accès libre,24/7,FALSE,FRIONE4403,2023-03-21,2023-12-14T14:56:35.115000+00:00
987,29316,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440301,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440301,2024-01-15,2024-01-16T00:00:17.176000+00:00
988,29317,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440302,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440302,2024-01-15,2024-01-16T00:00:17.176000+00:00
989,29318,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440303,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440303,2024-01-15,2024-01-16T00:00:17.176000+00:00
990,29319,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440304,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440304,2024-01-15,2024-01-16T00:00:17.176000+00:00
991,29320,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440305,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440305,2024-01-15,2024-01-16T00:00:17.176000+00:00
992,29321,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440306,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440306,2024-01-15,2024-01-16T00:00:17.176000+00:00
993,29322,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440307,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440307,2024-01-15,2024-01-16T00:00:17.176000+00:00
994,29323,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440308,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440308,2024-01-15,2024-01-16T00:00:17.176000+00:00
995,29324,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440309,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440309,2024-01-15,2024-01-16T00:00:17.176000+00:00


In [30]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,121,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2023-12-22,2024-01-15T07:39:10.104000+00:00
19,122,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2023-12-22,2024-01-15T07:39:10.104000+00:00
20,123,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2023-12-22,2024-01-15T07:39:10.104000+00:00
21,124,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2023-12-22,2024-01-15T07:39:10.104000+00:00
22,141,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2023-12-22,2024-01-15T07:39:10.104000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3232,79253,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2023-12-14T14:54:52.531000+00:00
3233,79262,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-12-14T14:54:52.296000+00:00
3234,79263,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-12-14T14:54:52.296000+00:00
3235,79264,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-12-14T14:54:52.296000+00:00


In [31]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3233,79262,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-12-14T14:54:52.296000+00:00
3234,79263,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-12-14T14:54:52.296000+00:00
3235,79264,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-12-14T14:54:52.296000+00:00
3236,79265,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2023-12-14T14:54:52.296000+00:00


In [32]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3233,79262,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-12-14T14:54:52.296000+00:00
3235,79264,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-12-14T14:54:52.296000+00:00


In [33]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3234,79263,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-12-14T14:54:52.296000+00:00
3236,79265,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2023-12-14T14:54:52.296000+00:00


- exemple : Stations Ouest charge (double saisie par deux opérateurs)

In [34]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1248,51394,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430012,2030-04-12,2023-12-14T14:59:16.945000+00:00
1419,51597,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22311,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22311,2023-04-07,2023-12-14T14:54:30.353000+00:00
1420,51598,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22312,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22312,2023-04-07,2023-12-14T14:54:30.353000+00:00


### Cohérence station - coordonnées
- 323 pdc sont associés à des stations avec plusieurs coordonnées (dont 127 Freshmile)

In [49]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][300:315]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1086,35253,web@freshmile.com,Freshmile,"[0.08745032, 48.40887156]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ERVTJ2,2024-01-09,2024-01-10T07:42:48.874000+00:00
1087,35259,web@freshmile.com,Freshmile,"[2.60894067, 49.3181843]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ERWKR1,2024-01-09,2024-01-10T07:42:48.874000+00:00
1088,35264,web@freshmile.com,Freshmile,"[0.088952, 48.398951]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ERXDZ1,2024-01-09,2024-01-10T07:42:48.874000+00:00
1089,35265,web@freshmile.com,Freshmile,"[0.088952, 48.398951]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ERXDZ2,2024-01-09,2024-01-10T07:42:48.874000+00:00
1090,35364,web@freshmile.com,Freshmile,"[0.088952, 48.398951]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ESRQM1,2024-01-09,2024-01-10T07:42:48.874000+00:00
1091,35365,web@freshmile.com,Freshmile,"[0.088952, 48.398951]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ESRQM2,2024-01-09,2024-01-10T07:42:48.874000+00:00
1092,35366,web@freshmile.com,Freshmile,"[0.088952, 48.398951]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ESRTS1,2024-01-09,2024-01-10T07:42:48.874000+00:00
1093,35367,web@freshmile.com,Freshmile,"[0.088952, 48.398951]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2ESRTS2,2024-01-09,2024-01-10T07:42:48.874000+00:00
1094,35438,web@freshmile.com,Freshmile,"[2.6089631, 49.31839082]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ETEPK1,2024-01-09,2024-01-10T07:42:48.874000+00:00
1095,35439,web@freshmile.com,Freshmile,"[2.6089631, 49.31839082]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ETEPK2,2024-01-09,2024-01-10T07:42:48.874000+00:00


In [51]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRLE2PYSCJHWOOBD', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
949,34040,web@freshmile.com,Freshmile,"[2.60894067, 49.3181843]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ECLPS1,2024-01-09,2024-01-10T07:42:48.874000+00:00
961,35259,web@freshmile.com,Freshmile,"[2.60894067, 49.3181843]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ERWKR1,2024-01-09,2024-01-10T07:42:48.874000+00:00
968,35438,web@freshmile.com,Freshmile,"[2.6089631, 49.31839082]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ETEPK1,2024-01-09,2024-01-10T07:42:48.874000+00:00
969,35439,web@freshmile.com,Freshmile,"[2.6089631, 49.31839082]",Avenue d'Auvelais 60700 Pont-Sainte-Maxence,FRLE2PYSCJHWOOBD,"E. Leclerc, Pont-Sainte-Maxence",Voirie,6,Accès libre,24/7,false,FRLE2ETEPK2,2024-01-09,2024-01-10T07:42:48.874000+00:00


In [48]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
355,19341,web@freshmile.com,Freshmile,"[1.391725, 43.566937]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE1,2022-11-09,2024-01-10T07:42:48.874000+00:00
356,19342,web@freshmile.com,Freshmile,"[0.66337, 43.85402]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE2,2022-11-09,2024-01-10T07:42:48.874000+00:00
374,21679,web@freshmile.com,Freshmile,"[1.523894, 44.147449]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF1,2022-11-09,2024-01-10T07:42:48.874000+00:00
375,21680,web@freshmile.com,Freshmile,"[4.636316, 44.26687]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF2,2022-11-09,2024-01-10T07:42:48.874000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [37]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-12-14T14:54:22.394000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-12-14T14:54:22.394000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-12-14T14:54:22.394000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-12-14T14:54:22.394000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-12-14T14:54:22.394000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-12-14T14:54:22.394000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-12-14T14:54:22.394000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-12-14T14:54:22.394000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-12-14T14:54:22.394000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-12-14T14:54:22.394000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [38]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
52,341,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-12-14T14:56:18.422000+00:00
53,342,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-12-14T14:56:18.422000+00:00
54,343,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-12-14T14:56:18.422000+00:00
55,344,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-12-14T14:56:18.422000+00:00
56,345,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-12-14T14:56:18.422000+00:00
57,346,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-12-14T14:56:18.422000+00:00


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [39]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.658987, 48.542091]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1852,52836,support@alizecharge.fr,Ouest Charge,"[-4.658987, 48.542091]",Rue Auguste Caroff 29830 PLOUDALMEZEAU,FRS29P29178001,Ploudalmezeau-Rue Auguste Caroff,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E291780012,2049-04-12,2023-12-14T14:59:16.945000+00:00
1989,53089,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912011,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912011,2023-04-14,2023-12-14T14:54:30.637000+00:00
1990,53090,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912012,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912012,2023-04-14,2023-12-14T14:54:30.637000+00:00


In [40]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1802,52720,support@alizecharge.fr,Ouest Charge,"[-4.374469, 48.471878]",Route de Plabennec 29860 KERSAINT PLABENNEC,FRS29P29095001,Kersaint Plabennec-Route de Plabennec,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E290950012,2052-05-29,2023-12-14T14:59:16.945000+00:00
2283,53385,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296211,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296211,2023-04-14,2023-12-14T14:54:30.637000+00:00
2284,53386,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296212,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296212,2023-04-14,2023-12-14T14:54:30.637000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [41]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2221,61182,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2024-01-10,2024-01-12T07:39:43.222000+00:00
2222,61183,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2024-01-10,2024-01-12T07:39:43.222000+00:00
2223,61184,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2024-01-10,2024-01-12T07:39:43.222000+00:00
2224,61185,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2024-01-10,2024-01-12T07:39:43.222000+00:00


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [42]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1190,47233,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200011,2024-01-09,2024-01-10T07:42:40.311000+00:00
1191,47234,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200012,2024-01-09,2024-01-10T07:42:40.311000+00:00
1192,47235,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200013,2024-01-09,2024-01-10T07:42:40.311000+00:00
1193,47236,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200014,2024-01-09,2024-01-10T07:42:40.311000+00:00
1194,47237,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60011,2024-01-09,2024-01-10T07:42:40.311000+00:00
1195,47238,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60012,2024-01-09,2024-01-10T07:42:40.311000+00:00
1196,47239,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60013,2024-01-09,2024-01-10T07:42:40.311000+00:00
1197,47240,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60021,2024-01-09,2024-01-10T07:42:40.311000+00:00
1198,47241,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60022,2024-01-09,2024-01-10T07:42:40.311000+00:00
1199,47242,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60023,2024-01-09,2024-01-10T07:42:40.311000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [43]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
320,13339,web@freshmile.com,Freshmile,"[2.1297, 48.82966]",Carr de l'Europe 78150 Le Chesnay-Rocquencourt,FRECHPNN4XSWH5DX,"Le Chesnay-Rocquencourt , Carr de l'Europe",Voirie,6,Accès libre,24/7,false,FRECHEAQJY1,2024-01-09,2024-01-10T07:42:48.874000+00:00
321,13340,exploitation@easycharge-vinci.COM,Easycharge services,"[2.1297, 48.82966]",Carr de l'Europe 78150 Le Chesnay-Rocquencourt,FRECHPNN4XSWH5DX,"Le Chesnay-Rocquencourt , Carr de l'Europe",Voirie,6,Accès libre,24/7,false,FRECHEAQJY2,2024-01-09,2024-01-12T07:39:51.174000+00:00
324,13376,exploitation@easycharge-vinci.COM,Easycharge services,"[2.12831, 48.81617]",Rue de Versaille 78150 Le Chesnay-Rocquencourt,FRECHPDCQ0LAWQZE,"Le Chesnay-Rocquencourt , Rue de Versaille",Voirie,4,Accès libre,24/7,false,FRECHEBSAQ1,2024-01-09,2024-01-12T07:39:51.174000+00:00
325,13379,web@freshmile.com,Freshmile,"[2.12831, 48.81617]",Rue de Versaille 78150 Le Chesnay-Rocquencourt,FRECHPDCQ0LAWQZE,"Le Chesnay-Rocquencourt , Rue de Versaille",Voirie,4,Accès libre,24/7,false,FRECHEBSAQ2,2024-01-09,2024-01-10T07:42:48.874000+00:00
334,13510,exploitation@easycharge-vinci.COM,Easycharge services,"[2.033491, 47.905369]",Biocoop 45430 Chécy,FRECHPKR0Q8THZQY,"Chécy , Biocoop 1",Voirie,6,Accès libre,24/7,false,FRECHEFTUE2,2024-01-09,2024-01-12T07:39:51.174000+00:00
335,13519,web@freshmile.com,Freshmile,"[2.033491, 47.905369]",Biocoop 45430 Chécy,FRECHPKR0Q8THZQY,"Chécy , Biocoop 1",Voirie,6,Accès libre,24/7,false,FRECHEFYZN1,2024-01-09,2024-01-10T07:42:48.874000+00:00
336,13521,web@freshmile.com,Freshmile,"[2.033491, 47.905369]",Biocoop 45430 Chécy,FRECHPKR0Q8THZQY,"Chécy , Biocoop 1",Voirie,6,Accès libre,24/7,false,FRECHEFYZN2,2024-01-09,2024-01-10T07:42:48.874000+00:00
337,13523,web@freshmile.com,Freshmile,"[2.033491, 47.905369]",Biocoop 45430 Chécy,FRECHPKR0Q8THZQY,"Chécy , Biocoop 1",Voirie,6,Accès libre,24/7,false,FRECHEFYZN3,2024-01-09,2024-01-10T07:42:48.874000+00:00
350,13775,web@freshmile.com,Freshmile,"[2.1297, 48.82966]",Carr de l'Europe 78150 Le Chesnay-Rocquencourt,FRECHPNN4XSWH5DX,"Le Chesnay-Rocquencourt , Carr de l'Europe",Voirie,6,Accès libre,24/7,false,FRECHERYFQ1,2024-01-09,2024-01-10T07:42:48.874000+00:00
351,13779,web@freshmile.com,Freshmile,"[2.1297, 48.82966]",Carr de l'Europe 78150 Le Chesnay-Rocquencourt,FRECHPNN4XSWH5DX,"Le Chesnay-Rocquencourt , Carr de l'Europe",Voirie,6,Accès libre,24/7,false,FRECHERYFQ3,2024-01-09,2024-01-10T07:42:48.874000+00:00


In [44]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
16400,17228,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2023-12-14T14:59:38.118000+00:00
16402,17230,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2023-12-14T14:59:38.118000+00:00
16404,17232,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2023-12-14T14:59:38.118000+00:00
17132,17960,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2023-12-14T15:00:52.063000+00:00
17133,17961,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2023-12-14T15:00:52.063000+00:00
17134,17962,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2023-12-14T15:00:52.063000+00:00


In [45]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
16400,17228,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2023-12-14T14:59:38.118000+00:00
16402,17230,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2023-12-14T14:59:38.118000+00:00
17132,17960,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2023-12-14T15:00:52.063000+00:00
17133,17961,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2023-12-14T15:00:52.063000+00:00
20140,20968,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2023-12-14T15:01:03.035000+00:00
20142,20970,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2023-12-14T15:01:03.035000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [46]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [47]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
75040,77016,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2023-12-14T14:57:07.770000+00:00
75041,77017,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2023-12-14T14:57:07.770000+00:00
75042,77018,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2023-12-14T14:57:07.770000+00:00
75043,77019,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2023-12-14T14:57:07.770000+00:00
75044,77020,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2023-12-14T14:57:07.770000+00:00
75045,77021,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2023-12-14T14:57:07.770000+00:00
75046,77022,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2023-12-14T14:57:07.770000+00:00
